#0. Default Setting

Executed in Colab environment.

* ML Framework
   - Python 3.7.10
   - Pytorch 1.8.1

* Hardware
   - RAM: 12.7G 
   - CPU: Intel(R) Xeon(R) CPU @ 2.20GHz (1core)

Assumed that data exists like below.
you also need a etri openapi key.

```
/content/gdrive/My Drive/data
├── 1_bert_download_001_bert_morp_pytorch.zip
```

Project Tree (directory only)
```
/content/KoBertSum
├── bert_data
├── json_data
├── logs
├── models
├── raw_data
├── results
├── src
│   ├── models
│   ├── others
│   └── prepro
└── urls
```

#1. Install dependency packages

In [1]:
# # install bheinzerling's pyrouge
# !git clone https://github.com/bheinzerling/pyrouge
# %cd pyrouge
# !python setup.py -q install
# # install missing dependency
# !apt install -q libxml-parser-perl
# %cd pyrouge
# !git clone https://github.com/andersjo/pyrouge.git rouge
# !pyrouge_set_rouge_path '/content/pyrouge/rouge/tools/ROUGE-1.5.5'
# %cd /content/pyrouge/rouge/tools/ROUGE-1.5.5/data
# !mv WordNet-2.0.exc.db WordNet-2.0.exc.db.orig
# !perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

In [ ]:
# # 기타 패키지 설치
!pip install -q pytorch_pretrained_bert
!pip install -q tensorboardX
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers

#Google Drive Mount

#3. BERT forward propagation workflow

In [27]:
"""
    Main training workflow
"""
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time
import numpy as np

import torch
from pytorch_pretrained_bert import BertConfig


import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import Summarizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
from others.logging import logger
# from models.trainer import build_trainer
# build_trainer의 dependency package pyrouge.utils가 import되지 않아 직접 셀에 삽입
from others.logging import logger, init_logger
import easydict

args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'summary',
    "bert_data_path":'../bert_data/korean',
    "model_path":'../models/bert_classifier_test',
    "bert_model":'../../korBert_models/001_bert_morp_pytorch',
    "result_path":'../results/korean',
    "temp_dir":'.',
    "bert_config_path":'../../korBert_models/001_bert_morp_pytorch/bert_config.json',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'../logs/bert_classifier',
    "test_from":'../models/bert_classifier_test/model_step_1000.pt'
})


def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        logger.info('* number of parameters: %d' % n_params)

    return trainer
class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summary(self, test_iter, step, cal_lead=False, cal_oracle=False):
        """ Validate model.
            valid_iter: validate data iterator
        Returns:
            :obj:`nmt.Statistics`: validation loss statistics
        """
        # Set model in validating mode.
        def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

        def _block_tri(c, p):
            tri_c = _get_ngrams(3, c.split())
            for s in p:
                tri_s = _get_ngrams(3, s.split())
                if len(tri_c.intersection(tri_s))>0:
                    return True
            return False

        if (not cal_lead and not cal_oracle):
            self.model.eval()
        stats = Statistics()

        
        with torch.no_grad():
            for batch in test_iter:
                src = batch.src
                labels = batch.labels
                segs = batch.segs
                clss = batch.clss
                mask = batch.mask
                mask_cls = batch.mask_cls


                gold = []
                pred = []

                if (cal_lead):
                    selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
                elif (cal_oracle):
                    selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                    range(batch.batch_size)]
                else:
                    sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

                    # loss = self.loss(sent_scores, labels.float())
                    # loss = (loss * mask.float()).sum()
                    # batch_stats = Statistics(float(loss.cpu().data.numpy()), len(labels))
                    # stats.update(batch_stats)

                    sent_scores = sent_scores + mask.float()
                    sent_scores = sent_scores.cpu().data.numpy()
                    selected_ids = np.argsort(-sent_scores, 1)
                # selected_ids = np.sort(selected_ids,1)
                

        return selected_ids


    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

def summary(args, b_list, device_id, pt, step):

    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    if (pt != ''):
        test_from = pt
    else:
        test_from = args.test_from
    logger.info('Loading checkpoint from %s' % test_from)
    checkpoint = torch.load(test_from, map_location=lambda storage, loc: storage)
    opt = vars(checkpoint['opt'])
    for k in opt.keys():
        if (k in model_flags):
            setattr(args, k, opt[k])
    print(args)

    config = BertConfig.from_json_file(args.bert_config_path)
    model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
    model.load_cp(checkpoint)
    model.eval()

    test_iter =data_loader.Dataloader(args, _lazy_dataset_loader(b_list),
                                  args.batch_size, device,
                                  shuffle=False, is_test=True)
    trainer = build_trainer(args, device_id, model, None)
    result = trainer.summary(test_iter,step)
    return result
def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

init_logger(args.log_file)
device = "cpu" if args.visible_gpus == '-1' else "cuda"
device_id = 0 if device == "cuda" else -1
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

#4. Input data morp-tokenization workflow

##your openapi_key

In [28]:
openapi_key = '220c9e91-8f30-442a-a8a5-5dffc2aab0c6'

##workflow

In [29]:
import argparse
import json
import os
import time
import urllib3
from glob import glob
import collections
import six
import gc

def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    requestJson = { "access_key": openapi_key, "argument": { "text": text, "analysis_code": "morp" } }
    http = urllib3.PoolManager()
    response = http.request( "POST", openApiURL, headers={"Content-Type": "application/json; charset=UTF-8"}, body=json.dumps(requestJson))
    
    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result
class BertData():
    def __init__(self, vocab_file_path):
        self.tokenizer = Tokenizer(vocab_file_path)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [''.join(s) for s in src]


        idxs = [i for i, s in enumerate(src) if (len(s) > 0)]

        src = [src[i][:20000] for i in idxs]
        src = src[:10000]

        if (len(src) < 3):
            return None

        src_txt = [''.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = text.split(' ')
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        tgt_txt = None
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")
class Tokenizer(object):
    def __init__(self, vocab_file_path):
        self.vocab_file_path = vocab_file_path
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        index = 0
        with open(self.vocab_file_path, "r", encoding='utf-8') as reader:

            while True:
                token = convert_to_unicode(reader.readline())
                if not token:
                    break

          ### joonho.lim @ 2019-03-15
                if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :

                    continue
                token = token.split('\t')[0].strip('_')

                token = token.strip()
                vocab[token] = index
                index += 1
        self.vocab = vocab
    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            try:
                ids.append(self.vocab[token])
            except:
                ids.append(1)
        if len(ids) > 10000:
            raise ValueError(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), 10000)
            )
        return ids
def _lazy_dataset_loader(pt_file):
    
    dataset = pt_file
    
    yield dataset
def News_to_input(text, openapi_key):
    newstemp = do_lang(openapi_key,text)
    news = newstemp.split(' ./SF ')[:-1]
    bertdata = BertData('../../korBert_models/001_bert_morp_pytorch/vocab.korean_morp.list')
    tmp = bertdata.preprocess(news)
    b_data_dict = {"src":tmp[0],
               "labels":[0,1,2],
               "segs":tmp[2],
               "clss":tmp[3],
               "src_txt":tmp[4],
               "tgt_txt":'hehe'}
    b_list = []
    b_list.append(b_data_dict) 
    return b_list

In [60]:
test = """저는 올해 초등학교 6학년인 남자아이와 4학년인 여자아이를 자녀로 두고 있는 42살의 14년차 전업주부 OOO입니다.
미국인과 결혼하신 이모 덕에 어려서부터 영어에 대한 관심이 많았고 영어공부도 좋아했습니다. 관련해서 더 깊이있게 공부하고 싶었지만 고3 시절 가정형편이 좋지 못한 사정으로 대학진학을 통한 공부의 꿈을 미루고 바로 취직을 하게 됐습니다. 이후 전업주부가 되기 전까지는 백화점에서 5년간 근무하며 판매왕, 친절왕으로 여러 번 뽑혔을 정도로 제 성격은 매사에 끈기가 있고 적극적이며, 주위 사람에게 친절한 편입니다.
하루 대부분의 시간을 주로 남편과 아이들을 위해 사용하지만 평일 낮 시간, 주말시간을 활용해 틈틈이 새로운 일을 배우고 자기계발을 하려고 노력합니다. 이 결과 작년에는 앙금플라워자격증, 올해에는 네일아트자격증을 취득하였습니다. 아주 대단한 자격증도 아니고 당장 어떤 가게를 차릴 것도 아니지만 무언가 새로운 일에 도전하고 성취감을 느끼는 걸 즐깁니다.
14년간 아내로서 엄마로서 최선을 다해 열심히 살아오면서도 항상 배움, 그중에서도 체계적인 학습에 대한 아쉬움이 있었습니다. 또한 주부이지만 성장하는 아이들에게 멋진 엄마의 모습을 보여주고 싶습니다.
제 환경이나 상황이 한 번에 긴 시간을 공부하기는 어렵겠지만 낮에는 아이들과 함께 공부하고, 아이들이 잠든 후 밤 시간을 이용해 짬짬이 공부하려고 합니다. 또한 주말에는 사이버외대에서 진행하는 특강이나 모임에도 적극적으로 참석해서 다른 학생들과 함께 공부하고, 쉽지는 않겠지만 해외 문화탐방과 같은 문화체험도 참가해 보고 싶습니다.
사이버외대를 졸업한 후에 여건이 된다면 대학원에도 진학해 보고 싶습니다. 학부과정에서 키운 언어에 대한 실력과 전문성을 바탕으로 대학원에서는 효과적으로 영어를 가르칠 수 있는 교육법(교수법)에 대해 공부할 계획입니다. 또한 향후 가능하다면 어린이, 학생을 대상으로 영어를 가르치는 봉사활동도 해보고 싶습니다.
원어민 교수님을 포함한 외국인과 거리낌 없이 소통하고 즐길 수 있는 가까운 미래 제 모습을 상상하며 열심히 하겠습니다.
"""

In [57]:
chat_history_ids

array([[0, 2, 5, 3, 4, 9, 7, 6, 1, 8]])

#5. html for SummaryBot

In [64]:
list(map(lambda x: x.strip(),test.split('.')))

['저는 올해 초등학교 6학년인 남자아이와 4학년인 여자아이를 자녀로 두고 있는 42살의 14년차 전업주부 OOO입니다',
 '미국인과 결혼하신 이모 덕에 어려서부터 영어에 대한 관심이 많았고 영어공부도 좋아했습니다',
 '관련해서 더 깊이있게 공부하고 싶었지만 고3 시절 가정형편이 좋지 못한 사정으로 대학진학을 통한 공부의 꿈을 미루고 바로 취직을 하게 됐습니다',
 '이후 전업주부가 되기 전까지는 백화점에서 5년간 근무하며 판매왕, 친절왕으로 여러 번 뽑혔을 정도로 제 성격은 매사에 끈기가 있고 적극적이며, 주위 사람에게 친절한 편입니다',
 '하루 대부분의 시간을 주로 남편과 아이들을 위해 사용하지만 평일 낮 시간, 주말시간을 활용해 틈틈이 새로운 일을 배우고 자기계발을 하려고 노력합니다',
 '이 결과 작년에는 앙금플라워자격증, 올해에는 네일아트자격증을 취득하였습니다',
 '아주 대단한 자격증도 아니고 당장 어떤 가게를 차릴 것도 아니지만 무언가 새로운 일에 도전하고 성취감을 느끼는 걸 즐깁니다',
 '14년간 아내로서 엄마로서 최선을 다해 열심히 살아오면서도 항상 배움, 그중에서도 체계적인 학습에 대한 아쉬움이 있었습니다',
 '또한 주부이지만 성장하는 아이들에게 멋진 엄마의 모습을 보여주고 싶습니다',
 '제 환경이나 상황이 한 번에 긴 시간을 공부하기는 어렵겠지만 낮에는 아이들과 함께 공부하고, 아이들이 잠든 후 밤 시간을 이용해 짬짬이 공부하려고 합니다',
 '또한 주말에는 사이버외대에서 진행하는 특강이나 모임에도 적극적으로 참석해서 다른 학생들과 함께 공부하고, 쉽지는 않겠지만 해외 문화탐방과 같은 문화체험도 참가해 보고 싶습니다',
 '사이버외대를 졸업한 후에 여건이 된다면 대학원에도 진학해 보고 싶습니다',
 '학부과정에서 키운 언어에 대한 실력과 전문성을 바탕으로 대학원에서는 효과적으로 영어를 가르칠 수 있는 교육법(교수법)에 대해 공부할 계획입니다',
 '또한 향후 가능하다면 어린이, 학생을 대상으로 영어를 가르치는 봉사활동도 해보고 싶

In [65]:
bot_input_ids = News_to_input(test, openapi_key)        
chat_history_ids = summary(args, bot_input_ids, -1, '', None)
pred_lst = list(chat_history_ids[0][:3])
final_text = ''
for i,a in enumerate(list(map(lambda x: x.strip(),test.split('.')))):
    if i in pred_lst:
        final_text = final_text+a+'. '
chat_history = test + '<token>' +final_text

[2021-11-26 16:25:34,102 INFO] Loading checkpoint from ../models/bert_classifier_test/model_step_1000.pt
[2021-11-26 16:25:34,851 INFO] loading archive file ../../korBert_models/001_bert_morp_pytorch
[2021-11-26 16:25:34,852 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



{'encoder': 'classifier', 'mode': 'summary', 'bert_data_path': '../bert_data/korean', 'model_path': '../models/bert_classifier_test', 'bert_model': '../../korBert_models/001_bert_morp_pytorch', 'result_path': '../results/korean', 'temp_dir': '.', 'bert_config_path': '../../korBert_models/001_bert_morp_pytorch/bert_config.json', 'batch_size': 1000, 'use_interval': True, 'hidden_size': 128, 'ff_size': 512, 'heads': 4, 'inter_layers': 2, 'rnn_size': 512, 'param_init': 0, 'param_init_glorot': True, 'dropout': 0.1, 'optim': 'adam', 'lr': 0.002, 'report_every': 1, 'save_checkpoint_steps': 5, 'block_trigram': True, 'recall_eval': False, 'accum_count': 1, 'world_size': 1, 'visible_gpus': '-1', 'gpu_ranks': [0], 'log_file': '../logs/bert_classifier', 'test_from': '../models/bert_classifier_test/model_step_1000.pt'}


[2021-11-26 16:25:36,817 INFO] * number of parameters: 109350145


gpu_rank 0


In [56]:
test

'\n이재명 더불어민주당 대선후보가 과거 조카가 저지른 살인 사건에 대한\u2008변호를 맡았던 일이 표면화되자 26일 "가슴 아픈 일이고 다시 한 번 사과드린다"고 밝혔다.\n\n이 후보는 이날 기자들과 만나 "모든 범죄의 피해자들은 억울한 것"이라면서도 "그 점에 대해서 제가 멀다고 할 수도 없는 친척의 일을 제가 처리하였는데 아쉬움, 억울함에 대해서 제가 말씀드렸다"고 했다. 이 후보는 거듭 "안타까운 일", "마음 아픈 일"이라면서 "다시 한 번 사과드린다"고 말했다.\n\n앞서 이 후보는 유엔의 여성 폭력 추방의 날이었던 지난 25일 자신의 페이스북을 통해 "제 일가 중 일인이 과거 데이트 폭력 중범죄를 저질렀는데, 그 가족들이 변호사를 선임할 형편이 못돼 일가 중 유일한 변호사인 제가 변론을 맡을 수밖에 없었다"며 "피해자와 유가족분들에게 깊은 위로와 사과의 말씀을 드린다"고 했다. 이어 "제게도 이 사건은 평생 지우지 못할 고통스런 기억이다. 어떤 말로도 피해자와 유족들의 상처가 아물지 않을 것"이라며 데이트 폭력에 대한 특별 대책을 강구하겠다고 밝힌 바 있다.\n\n이 후보가 언급한 \'데이트 폭력 중범죄\'는 지난 2006년 5월 서울 강동구에서 일어난 \'모녀 살인 사건\'이다. 당시 이 후보 조카 김 모씨는 헤어진 여자친구가 살던 집에 찾아가 전 여자친구와 그녀의 어머니를 흉기로 여러 차례 찔러 살해해 대법원에서 무기징역을 확정 받았다. 이 후보는 가해자인 조카의 1, 2심 재판 변호를 맡았고, 당시 조카를 변호하며 \'충동 조절 능력 저하로 심신미약 상태에 있었다\'며 감형을 주장한 것으로 전해졌다.\n\n해당 사건으로 딸과 아내를 잃은 A씨는 이날 <문화일보>와의 인터뷰에서 "15년이 지났지만 그 일만 생각하면 심장이 저릿저릿하다"며 "사건 당시에도 사과는 없었고, 현재까지도 이 후보 일가 측으로부터 사과 연락이 온 적이 단 한 번도 없다"고 했다. A씨는 "우리는 평생을 고통 속에서 살아가야 하는데, 이제 와서 예전 일을 끄집어내 보란

In [66]:
final_text.strip().split('\n')

['저는 올해 초등학교 6학년인 남자아이와 4학년인 여자아이를 자녀로 두고 있는 42살의 14년차 전업주부 OOO입니다. 이 결과 작년에는 앙금플라워자격증, 올해에는 네일아트자격증을 취득하였습니다. 14년간 아내로서 엄마로서 최선을 다해 열심히 살아오면서도 항상 배움, 그중에서도 체계적인 학습에 대한 아쉬움이 있었습니다.']

'\n김종인 전 국민의힘 비상대책위원장이 26일 윤석열 국민의힘 대선후보 총괄선거대책위원장을 맡지 않겠다는 의사를 밝혔다. 김병준 전 비상대책위원장이 상임선대위원장을 수락한 직후다.\n\n김병준 전 비상대책위원장은 이날 국민의힘 당사에서 기자간담회를 열고 상임선대위원장을 맡기로 했다고 밝혔다. 김병준 전 위원장은 "상임선대위원장직을 열심히 할 생각"이라고 밝혔다. 그는 "우리 정치가 시대에 뒤떨어져 과감히 바꿀 때가 됐다고 생각했는데, 그런 일을 하겠다는 분을 혼자 뛰게 둔다는 게 우리 모두의 도리가 아니라는 생각이 들었다. 무엇이든 돕겠다는 생각"이라고 밝혔다. 그동안 김종인 전 위원장과 불화로 돌던 상임선대위원장 사퇴설을 일축한 것이다.\n\n김병준 전 위원장이 상임선대위원장직을 수락한 직후 김종인 전 위원장은 \'총괄선대위원장을 고려하지 않겠다\'는 의사를 내비쳤다. 김종인 전 위원장은 서울 종로구 사무실에서 나온 뒤 \'김병준 상임위원장이 열심히 하겠다고 밝혔는데, 총괄선대위원장직은 고려하지 않는 건가\'라고 묻는 기자의 질문에 고개를 끄덕였다. \'끄덕이신 거 맞느냐\'고 재차 묻는 말에도 김종인 전 위원장은 다시 고개를 끄덕였다.\n\n그밖에 선대위 구성과 관련한 질문에 김종인 전 위원장은 "나는 아무 전달을 받은 게 없다. 할 말이 없다는데 자꾸 물어보느냐"고 답했다.\n\n김종인 전 위원장은 그동안 김병준 상임선대위원장 인선을 철회하는 조건으로 총괄선대위원장을 맡겠다는 의사를 밝혀왔다. 자신의 의사가 관철되지 않는 상황에서 총괄선대위원장을 맡지 않기로 결심한 것으로 해석된다.\n\n이로써 윤석열 후보 선대위는 총괄선대위원장 없이 김병준 상임총대위원장-김한길 새시대준비위원장 \'투 김\' 체제로 시작하거나, 새로운 총괄선대위원장을 영입할 가능성도 생겼다.\n\n이준석 국민의힘 대표는 김종인 전 위원장이 합류하지 못할 경우 다른 인사를 총괄선대위원장으로 세워야 한다는 뜻을 밝히기도 했다. 이 대표는 지난 25일 KBS라디오 \'최경영의 최강시사\'에

In [16]:
!pip install --upgrade dash

Looking in indexes: http://ftp.daumkakao.com/pypi/simple


In [18]:
dbc.InputGroupAddon()

AttributeError: InputGroupAddon was deprecated in dash-bootstrap-components version 1.0.0. You are using 1.0.0. For more details please see the migration guide: https://dbc-v1.herokuapp.com/migration-guide/

In [38]:
import time

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

def textbox(text, box="other"):
    style = {
        "max-width": "55%",
        "width": "max-content",
        "padding": "10px 15px",
        "border-radius": "25px"
    }

    if box == "self":
        style["margin-left"] = "auto"
        style["margin-right"] = 0

        color = "primary"
        inverse = True

    elif box == "other":
        style["margin-left"] = 0
        style["margin-right"] = "auto"

        color = "light"
        inverse = False

    else:
        raise ValueError("Incorrect option for `box`.")

    return dbc.Card(text, style=style, body=True, color=color, inverse=inverse)

conversation = html.Div(
    style={
        "width": "80%",
        "max-width": "800px",
        "height": "70vh",
        "margin": "auto",
        "overflow-y": "auto",
    },
    id="display-conversation",
)

controls = dbc.InputGroup(
    style={"width": "80%", "max-width": "800px", "margin": "auto"},
    children=[
        dbc.Input(id="user-input", placeholder="Write to the chatbot...", type="text"),
        dbc.InputGroupAddon(dbc.Button("Submit", id="submit"), addon_type="append",),
    ],
)


# Define app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Define Layout
app.layout = dbc.Container(
    fluid=True,
    style={'background-image': 'url(https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile21.uf.tistory.com%2Fimage%2F99A30D4B5CB15385210EA0)'},
    children=[
        html.H1("뉴스뚝딱"),
        html.Hr(),
        dcc.Store(id="store-conversation", data=""),
        conversation,
        controls
    ],
)
@app.callback(
    Output("display-conversation", "children"), [Input("store-conversation", "data")]
)
def update_display(chat_history):
    return [
        textbox(x, box="self") if i % 2 == 0 else textbox(x, box="other")
        for i, x in enumerate(chat_history.split('<token>'))
    ]

@app.callback(
    [Output("store-conversation", "data"), Output("user-input", "value")],
    [Input("submit", "n_clicks"), Input("user-input", "n_submit")],
    [State("user-input", "value"), State("store-conversation", "data")],
)
def run_chatbot(n_clicks, n_submit, user_input, chat_history):
    if n_clicks == 0:
        return "", ""

    if user_input is None or user_input == "":
        return chat_history, ""

    bot_input_ids = News_to_input(chat_history + user_input, openapi_key)
        
    chat_history_ids = summary(args, bot_input_ids, -1, '', None)
    pred_lst = list(chat_history_ids[0][:5])
    final_text = ''
    for i,a in enumerate(user_input.split('. ')):
        if i in pred_lst:
            final_text = final_text+a+'. '
    chat_history = user_input + '<token>' +final_text

    return chat_history, ""

AttributeError: InputGroupAddon was deprecated in dash-bootstrap-components version 1.0.0. You are using 1.0.0. For more details please see the migration guide: https://dbc-v1.herokuapp.com/migration-guide/

#6. RUN!

In [ ]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>